In [ ]:
import nltk
nltk.download('punkt')
from pyzotero import zotero
import numpy as np
import collections
import requests
import collections
import litstudy #Use pip install git+https://github.com/NLeSC/litstudy to download dev version. Other encoding problem when loading ris files (load_ris_file needs to use robust_open instead of open)
import os
import numpy as np
import pandas as pd
from pathlib import Path
import pickle
import re
import shutil
from pyzotero import zotero
#import cProfile
import re
#import itertools as it
from inspect import getsourcefile


In [ ]:
#Read and compile references from the WoS search
#into a single document set (lit_study format)
def rpickle_bibdocset(in_dirpath, in_pattern, out_pickle):
    if not out_pickle.exists():
        #Get list of every bib file
        bib_initlist = [p for p in list(in_dirpath.glob('*')) 
                        if re.compile(in_pattern).match(str(p))]
        #Read bib files from first scoping and join them (takes ~15-20 sec/1000 refs)
        reflist = []
        for bib in bib_initlist:
            reflist += litstudy.load_bibtex(bib)

        #Pickle them (save the full document set as a binary file on disk that can be easily retrieved)
        with open(out_pickle, 'wb') as f:
            pickle.dump(reflist, f)
    else:
        #Read pre-saved document set
        with open(out_pickle, 'rb') as f:
            reflist =  pickle.load(f)
    return(reflist)
            
#Get titles and dois from zotero test list
def get_testlist(library_id, api_key_path):
    api_key = api_key_path.read_text().strip() 
    zot = zotero.Zotero(library_id = library_id, library_type = 'group', api_key = api_key) #Get 
    testlist_colID = str([col['key'] for col in zot.collections_top()
                          if col['data']['name'] == 'test list'][0])
    testlist_items = zot.everything(zot.collection_items_top(testlist_colID))

    testlist_title_dois = collections.defaultdict(list)
    for ref in testlist_items:
        testlist_title_dois[ref['key']].append(ref['data']['title'])
        if 'DOI' in ref['data']:
            testlist_title_dois[ref['key']].append(ref['data']['DOI'])
        else:
            testlist_title_dois[ref['key']].append(np.nan)
        #testlist_title_dois[ref['key']].append(ref['data']['itemType'])
    return(testlist_title_dois)

#Get all dois and titles in references returned from search
def tabulate_searchlist(in_reflist, out_csvpath):
    if not out_csvpath.exists():
        reflist_dict = {}
        for i, ref in enumerate(in_reflist):
            reflist_dict[i] = [re.sub("[^a-zA-Z\d\s]", "", ref.title.replace('\n', ' ').lower()),
                               ref.publication_source, ref.publication_year, ref.abstract]
            if 'doi' in ref.entry:
                reflist_dict[i].append(ref.entry['doi'])
            else:
                reflist_dict[i].append(np.nan)

        reflist_pd = pd.DataFrame.from_dict(reflist_dict, orient='index')
        reflist_pd.columns = ['title', 'source', 'year', 'abstract', 'doi']

        reflist_pd.to_csv(out_csvpath)
    else:
        reflist_pd = pd.read_csv(out_csvpath)
    return(reflist_pd)

In [ ]:
def write(x,y):
    with open(x,'a') as f:
        f.write(y)
        f.write('\n')
        f.close()
    return _

In [ ]:
# def findNear(text,word1,word2,precede=False):
#     wa = re.findall(rf"{word1}" +r"\W+(?:\w+\W+){0,0}?" + rf"{word2}" + r"\b",prepped_text)
#     if precede==False:
#         wa1 = re.findall(rf"{word2}" +r"\W+(?:\w+\W+){0,0}?" + rf"{word1}" + r"\b",prepped_text)
#         return wa, wa1
#     else:
#         return wa

def findSimp(text,word1,word2,precede=False):
    '''
    precede = False means word1 and word2 are looked at with either being first word
    precede = True means word1 must be first, word2 must be second
    '''

    regexp = f"{word1}\\W{word2}\\b"
    if precede==False:
        regexp = f"({regexp})|({word2}\\W{word1}\\b)"

    wa = re.findall(regexp, text)
    wal = len(wa)

    return(wal)


def findSuperSimp(text,word):
    return len(re.findall(f"{word}", text))

In [ ]:
#Join all strings in a list with | signs and parentheses
def recomb(in_str):
    if isinstance(in_str, list):
        return(f"({'|'.join(f'({w})' for w in in_str)})")
    else:
        return(in_str)

#Profile whether faster to use product upstream or to use | in regex
# with cProfile.Profile() as prof:
#     for x in range(100000):
#         x1 = list(it.product(combo1_1, combo1_2))
#         for j in x1:
#             findSimp(prepped_text, j[0], j[1],precede=False)
# prof.print_stats()
#
# with cProfile.Profile() as prof:
#     x1 = [recomb(combo1_1), recomb(combo1_2)]
#     for x in range(100000):
#         findSimp(prepped_text,x1[0],x1[1],precede=False)
# prof.print_stats()
###### Using | in regex exp is 6.5 faster

def combo_refind(in_searchdict, text):
    for regexp_combo in in_searchdict.values():
        if regexp_combo[0] == 'with':
            k = findSimp(text, regexp_combo[1][0], regexp_combo[1][1], precede=False)
        elif regexp_combo[0] == 'pre':
            k = findSimp(text, regexp_combo[1][0], regexp_combo[1][1], precede=True)
        elif regexp_combo[0] is None:
            k = findSuperSimp(text, regexp_combo)
        else:
            break

        return (k)
        # if k > 0:
        #
        #     break

In [ ]:
z = []

with open('archive2022/zotero_keys.txt') as f:
    for idx,x in enumerate(f):
        y = x.split(' ')
        z.append(y[-1].strip())
        # print(idx,y)
        if idx == 2:
            print('\n')
            break

# print(z)

In [ ]:
### 
### use this to configure with your own api key and groupnum.

###  https://pyzotero.readthedocs.io/en/latest/#getting-started-short-version

group_id = z[1]
api_key = z[2][1:-1]

zot = zotero.Zotero(group_id,'group',api_key)
cols = zot.collections()

# itz = zot.everything(zot.collection_items(cols[-1]['key'])) 
#not helpful here, includes multiple entries of same

itz1 = zot.everything(zot.collection_items_top(cols[-1]['key']))

In [ ]:
#Get title, abstract, and tags
A = []
for idx,_ in enumerate(range(len(itz1))):
    x = itz1[idx]['data']['title']
    y = itz1[idx]['data']['abstractNote']
    z = [i['tag'] for i in itz1[idx]['data']['tags']]
    A.append([x,y,z])   

In [ ]:
def DOI_ngram(A):
    count0 = collections.Counter()
    s1 = A[0].replace("'", '')
    s2 = s1.replace("?", '')
    s3 = s2.replace(".",'')
    s4 = s3.replace(",",'')
    s5 = s4.replace(":",'')
    s6 = s5.lower()
    tokens = nltk.word_tokenize(s6)
    every = nltk.everygrams(tokens,2,4)
    count0 = count0 + (collections.Counter(every))
    count0 = count0.most_common()

    count1 = collections.Counter()
    s1 = A[1].replace("'", '')
    s2 = s1.replace("?", '')
    s3 = s2.replace(".",'')
    s4 = s3.replace(",",'')
    s5 = s4.replace(":",'')
    s6 = s5.lower()
    tokens = nltk.word_tokenize(s6)
    every = nltk.everygrams(tokens,2,4)
    count1 = count1 + (collections.Counter(every))
    count1 = count1.most_common()

    count2 = collections.Counter()
    for idx, i in enumerate(A[2]):
        x = collections.Counter([l.lower() for l in i])
        count2 += x
    count2 = count2.most_common()



    count3 = count0 + count1 + count2
    return count3

In [ ]:
count4 = []
badwords = ['the','in','of','for','and','an','as','no','to','(','we','by','from','was','are','than','have','this','has','is','that','these','on','be','or','at']
for c in A:
    d = DOI_ngram(c)
    for idx,i in enumerate(d):
        coun = 0
        for idj,j in enumerate(i[0]):
            if j in badwords:
                break
            else:
                coun+=1
        if coun == len(i[0]):
            count4.append(i)   

# y = DOI_ngram(A[0])
count5 = []
for idx,i in enumerate(count4):
    coun = 0
    for idj,j in enumerate(i[0]):
        if j in badwords:
            break
        elif len(j) == 1:
            break
        else:
            coun+=1
    if coun == len(i[0]):
        count5.append(i)   
count6 = dict(count5)

count7 = collections.Counter(count6)
count7.most_common()
# for w in sorted(count6,key=count6.get,reverse=True):
#     print(w,count6[w])

In [ ]:
def searching(
    query_string,
    broad='no',
    has_ngrams='true',
    does_date_matter = 'no',
    from_date='2010-01-01',
    to_date='2022-12-12'):

    if broad == 'no':
        if does_date_matter == 'yes':
            institution = requests.get(
                f'https://api.openalex.org/works?filter=abstract.search:{query_string},has_ngrams:{has_ngrams},from_publication_date:{from_date},to_publication_date:{to_date}&page=1&per-page=20'
            ).json()
        else:
            institution = requests.get(
                f'https://api.openalex.org/works?filter=title.search:{query_string},has_ngrams:{has_ngrams},&page=1&per-page=20'
            ).json()

    elif broad == 'yes':
        institution = requests.get(
            f'https://api.openalex.org/works?search={query_string}&page=1&per-page=20'
        ).json()
    return institution
    
    # a = []
    # for x in range(30):
    #     try:
    #         y = institution['results'][x]['ngrams_url']
    #         if x < 5:
    #             print(institution['results'][x]['title'])
    #             print(institution['results'][x]['authorships'][0]['author']['display_name'])
    #             print(institution['results'][x]['doi'])
    #             print(institution['results'][x]['relevance_score'])
    #             print(institution['results'][x]['publication_date'])
    #             print(y,'\n')  
    #         z = requests.get(y).json()
    #         a.append(z)
    #     except:
    #         pass

    # return a


count = 0
print(f"count of identical / position in list / total size of list")
listofngrams = []
for i in range(len(B)):
    _ngrams = searching(B[i][0],broad='yes')
    # print(_ngrams['meta'])
    if _ngrams['meta']['count'] == 0:
        continue
    else:
        y = _ngrams['results'][0]['title']
        if y == B[i][0]:
            count+=1
            print(f"{count} / {i} / {len(B)}")
            urlll = _ngrams['results'][0]['ngrams_url']
            z = requests.get(urlll).json()
            listofngrams.append(z)
            



# for idx,i in enumerate(_ngrams):
#     if idx == 0:
#         j = i['ngrams']
#         k = pd.DataFrame.from_dict(j)
#     else:
#         j = i['ngrams']
#         l = pd.DataFrame.from_dict(j)
#         k = pd.concat([k,l])

In [ ]:
import pandas as pd 

ngram = []

for i in listofngrams:
    for j in i['ngrams']:
        ngram.append((j['ngram'].lower(),j['ngram_count']))

print(len(ngram))

b = collections.Counter()
for ng,co in ngram:
    b[ng] += co

c = [[x,y] for x,y in b.items()]

badwords = ['add words to filter as necessary',' .$', 'et al','et$', '3 /s', 'm 3 /s']

df = pd.DataFrame.from_records(c,columns=['ngram','ngram_count'])
df['ngram_tokens'] = df['ngram'].str.split().str.len() #https://stackoverflow.com/questions/37483470/how-to-calculate-number-of-words-in-a-string-in-dataframe
df1 = df.dropna(axis=0)
df2 = df1.loc[(df1['ngram_tokens'] > 1) & (df1['ngram_tokens'] < 5)]
df3 = df2.sort_values(by=['ngram_count','ngram_tokens'],ascending=False)

for i in badwords:
    df3 = df3[~df3['ngram'].str.contains(i)]

df4 = df3.to_numpy()
df4.shape

df3.head(n=20)